With this model, the intent is to classify a song using its data as "stamina" or "tech" to add a feature to the app which removes user input and instead detects which rating model should be used instead.

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_confusion_matrix, precision_score, recall_score, accuracy_score, f1_score,\
                            roc_curve, auc, classification_report, log_loss, mean_squared_error, mean_squared_log_error
from sklearn.decomposition import PCA

from statsmodels.api import qqplot

from xgboost import XGBClassifier, XGBRegressor, plot_importance, plot_tree, DMatrix
import shap


import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
##display all dataframe columns
pd.set_option('display.max_columns', None)

stam = pd.read_csv("../data/stam.csv")
tech = pd.read_csv("../data/not_stam.csv")

In [26]:
import sys
sys.path.append("..")

from src.data_cleaning import data_cleaner

In [27]:
cleaned_tech = data_cleaner(tech)

Initialized Tech DataFrame with 4808 rows.

Removing outliers and filling NaN values...

The songs in this Tech dataset are up to 164.002 seconds (2.733 minutes) long.
The songs in this Tech dataset have up to 826 steps.
The songs in this Tech dataset have a max  bpm weighted average up to 175 bpm.
The songs in this Tech dataset have up to 21 bpm changes.
The songs in this Tech dataset have up to 6.387 NPS.

Returning cleaned Tech DataFrame with 3209 rows and 31 columns.


In [28]:
cleaned_stam = data_cleaner(stam, is_stamina = True)

Initialized Stamina DataFrame with 2926 rows.

Removing outliers and filling NaN values...

The songs in this Stamina dataset are up to 1695.086 seconds (28.251 minutes) long.
The songs in this Stamina dataset have up to 16804steps.
The songs in this Stamina dataset have a max  bpm weighted average up to 216 bpm.
The songs in this Stamina dataset have up to 4 bpm changes.
The songs in this Stamina dataset have up to 10.866 NPS.

Returning cleaned Stamina DataFrame with 2255 rows and 40 columns.


In [29]:
cleaned_tech['target'] = 0

In [30]:
cleaned_stam['target'] = 1

In [31]:
all_songs = pd.concat([cleaned_tech, cleaned_stam], ignore_index=True)

In [32]:
all_songs

,title,artist,song_seconds,step_count,measure_count,bpm_weighted_avg,bpm_max,bpm_min,bpm_mode,bpm_change_count,song_nps,nps_per_measure_max,nps_per_measure_avg,nps_per_measure_median,nps_per_measure_std,nps_per_measure_mode,jumps,hands,quads,holds,mines,rolls,crossovers,footswitches,crossover_footswitches,jacks,invalid_crossovers,stop_count,stream_total,difficulty,rating,target,stream_count,stream_size_max,stream_size_avg,stream_size_std,break_count,break_size_max,break_size_avg,break_total,break_size_std
0,666!,Getter & Ghastly,118.064516,623,61,124.0,124.0,124.0,124.0,0,5.276776,8.266667,5.276776,5.683333,2.513704,5.683333,12,0,0,5,39,5,21,58,12,5,7,0,14,Challenge,11.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,666!,Getter & Ghastly,118.064516,502,61,124.0,124.0,124.0,124.0,0,4.251913,6.200000,4.251913,4.650000,1.929489,5.166667,9,0,0,10,4,0,20,36,17,21,14,0,0,Hard,9.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,666!,Getter & Ghastly,118.064516,319,61,124.0,124.0,124.0,124.0,0,2.701913,4.133333,2.701913,3.100000,1.227717,3.100000,11,0,0,8,0,0,5,5,5,10,2,0,0,Medium,6.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,666!,Getter & Ghastly,118.064516,201,61,124.0,124.0,124.0,124.0,0,1.702459,2.583333,1.702459,2.066667,0.720525,2.066667,6,0,0,8,0,0,4,1,2,3,0,0,0,Easy,4.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,666!,Getter & Ghastly,118.064516,111,61,124.0,124.0,124.0,124.0,0,0.940164,2.066667,0.940164,1.033333,0.503657,1.033333,17,0,0,2,0,0,0,0,0,1,0,0,0,Beginner,2.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5459,Summertime,Lime,452.173913,3707,260,138.0,138.0,138.0,138.0,0,8.198173,9.200000,8.198173,9.200000,2.012178,9.200000,2,0,0,6,7,73,0,0,0,0,0,0,200,Challenge,13.0,1,2.0,120.0,100.000000,28.284271,3.0,58.0,20.000000,60.0,32.908965
5460,Summertime,Lime,452.173913,3228,260,138.0,138.0,138.0,138.0,0,7.138846,9.200000,7.138846,6.900000,2.290305,9.200000,16,0,0,37,23,13,0,0,0,0,0,0,119,Hard,12.0,1,8.0,32.0,14.875000,11.630471,9.0,58.0,15.666667,141.0,20.651876
5461,Summertime,Lime,452.173913,2774,260,138.0,138.0,138.0,138.0,0,6.134808,9.200000,6.134808,6.325000,2.000030,6.900000,18,0,0,67,23,13,0,0,0,0,0,0,44,Medium,11.0,1,7.0,28.0,6.285714,9.776064,8.0,91.0,27.000000,216.0,30.397368
5462,Summertime,Lime,452.173913,2356,260,138.0,138.0,138.0,138.0,0,5.210385,9.200000,5.210385,5.750000,1.892955,6.900000,20,0,0,72,23,14,0,0,0,0,0,0,10,Easy,10.0,1,3.0,7.0,3.333333,3.214550,4.0,139.0,62.500000,250.0,53.544374


In [33]:
#due to some columns being removed from the tech dataset for being irrelevant, I will fill the NaN values with 0

all_songs = all_songs.fillna(0)